In [11]:
import os
import sys
import geopandas
import geopandas as gpd
import pandas
%matplotlib notebook
import matplotlib.pyplot as plt

In [12]:
data_folder = "C:\\Users\\Lena\\Dropbox\\Hackathon\\rv"

### Load incoming data

In [34]:
electricity_cov_path = os.path.join(
data_folder, 'incoming_data', 'stluc_elec_coverage_area_polygon.shp')

electricity_path = os.path.join(
data_folder, 'data', 'infrastructure', 'electricity.shp')

buildings_path = os.path.join(
data_folder, 'data', 'buildings', 'stluc_buildings.shp')

buildings = geopandas.read_file(buildings_path)
electricity_coverage_area = geopandas.read_file(electricity_cov_path)
electricity = geopandas.read_file(electricity_path)

In [ ]:
#output_electricity = os.path.join(
#data_folder, 'data', 'infrastructure', 'electricity.shp')

### Assign electricity demand based on footprint

#### Total electricity demand => 397.8 GWh (from NISMOD LP)

In [14]:
buildings['ed_totals'] = 397.8


In [15]:
 # compute volume of each building
buildings['volume'] = buildings.footprint * buildings.height
buildings.head()


,id,height,footprint,geometry,ed_totals,volume
0,0,3.47,14.35580,"POLYGON ((514497.2324631526 1515576.104128955,...",397.8,49.814626
1,1,3.52,14.28525,"POLYGON ((514501.0824631522 1515582.024128955,...",397.8,50.284080
2,2,8.31,140.30920,"POLYGON ((514494.2124631524 1515581.334128955,...",397.8,1165.969452
3,3,4.88,102.64265,"POLYGON ((514387.5924631524 1515619.244128956,...",397.8,500.896132
4,4,3.63,8.61300,"POLYGON ((514425.6724631517 1515641.424128955,...",397.8,31.265190


In [16]:
# output csv to include building_id, type, service_demanded, unit, value(volume of building)
volume_totals = buildings.volume.sum()
buildings['volume_totals']= volume_totals
buildings['value'] = buildings.volume / buildings.volume_totals * buildings.ed_totals

buildings['service_demanded'] = 'electricity'
buildings['unit'] = 'GWh'
buildings = buildings[['id','service_demanded', 'unit', 'value', 'geometry']]

buildings.head()

,id,service_demanded,unit,value,geometry
0,0,electricity,GWh,0.000353,"POLYGON ((514497.2324631526 1515576.104128955,..."
1,1,electricity,GWh,0.000356,"POLYGON ((514501.0824631522 1515582.024128955,..."
2,2,electricity,GWh,0.008257,"POLYGON ((514494.2124631524 1515581.334128955,..."
3,3,electricity,GWh,0.003547,"POLYGON ((514387.5924631524 1515619.244128956,..."
4,4,electricity,GWh,0.000221,"POLYGON ((514425.6724631517 1515641.424128955,..."


In [25]:
coverage_buildings = geopandas.sjoin(electricity_coverage_area,buildings, how='inner', op= 'contains')
coverage_buildings = coverage_buildings.rename(columns={'id_left':'id'})
coverage_buildings.head()

,id,name,geometry,index_right,id_right,service_demanded,unit,value
0,2,union,"POLYGON ((509659.7335479773 1550131.548752962,...",45774,45774,electricity,GWh,0.001843
2,4,prasline,"POLYGON ((514647.9315231672 1543652.471453183,...",45774,45774,electricity,GWh,0.001843
0,2,union,"POLYGON ((509659.7335479773 1550131.548752962,...",45791,45791,electricity,GWh,0.002349
2,4,prasline,"POLYGON ((514647.9315231672 1543652.471453183,...",45791,45791,electricity,GWh,0.002349
0,2,union,"POLYGON ((509659.7335479773 1550131.548752962,...",45935,45935,electricity,GWh,0.009855


In [26]:
coverage_buildings = coverage_buildings.groupby(['id', 'unit', 'service_demanded']).sum()[['value']]

coverage_buildings.head()

,,,value
id,unit,service_demanded,
1,GWh,electricity,56.387692
2,GWh,electricity,74.809320
3,GWh,electricity,63.893964
4,GWh,electricity,48.871273
5,GWh,electricity,74.533876


In [36]:
electricity_cap = pandas.merge(electricity,coverage_buildings, on =['id'])

electricity_cap.head()

,id,name,type,capacity,unit,geometry,value
0,1,castries,substation,30.0,GWh,"POLYGON ((509228.6672421463 1548778.674672909,...",56.387692
1,4,prasline,substation,200.0,GWh,"POLYGON ((520224.3125255918 1534850.964799632,...",48.871273
2,6,soufriere,substation,20.0,GWh,"POLYGON ((503089.4458903986 1531546.229156021,...",23.484932
3,7,culdesac,main_station,100.0,GWh,"POLYGON ((507129.5149248281 1544928.839784743,...",56.111116
4,3,reduit,substation,30.0,GWh,"POLYGON ((513259.6180513942 1557195.106644747,...",63.893964


In [37]:
electricity_cap = electricity_cap[['id', 'name', 'type', 'unit', 'value', 'geometry']]
electricity_cap = electricity_cap.rename(columns={'value':'capacity'})

In [38]:
electricity.head()

,id,name,type,capacity,unit,geometry
0,1,castries,substation,30.0,GWh,"POLYGON ((509228.6672421463 1548778.674672909,..."
1,4,prasline,substation,200.0,GWh,"POLYGON ((520224.3125255918 1534850.964799632,..."
2,6,soufriere,substation,20.0,GWh,"POLYGON ((503089.4458903986 1531546.229156021,..."
3,7,culdesac,main_station,100.0,GWh,"POLYGON ((507129.5149248281 1544928.839784743,..."
4,3,reduit,substation,30.0,GWh,"POLYGON ((513259.6180513942 1557195.106644747,..."


In [40]:
electricity_cap.to_file(electricity_path)